<a href="https://colab.research.google.com/github/jingwen0/systemeAQ/blob/main/AQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import json

# ### 第一部分：对于训练数据的处理：读取文件和预处理

# - ```文本的读取```： 需要从文本中读取数据，此处需要读取的文件是```dev-v2.0.json```，并把读取的文件存入一个列表里（list）
# - ```文本预处理```： 对于问题本身需要做一些停用词过滤等文本方面的处理
# - ```可视化分析```： 对于给定的样本数据，做一些可视化分析来更好地理解数据


# #### 1.1节： 文本的读取
# 把给定的文本数据读入到```qlist```和```alist```当中，这两个分别是列表，其中```qlist```是问题的列表，```alist```是对应的答案列表

In [7]:
!mkdir data
!wget -P data https://raw.githubusercontent.com/jingwen0/systemeAQ/main/piaf-v1.2.json

mkdir: cannot create directory ‘dat’: File exists
--2022-06-04 19:53:46--  https://raw.githubusercontent.com/jingwen0/systemeAQ/main/piaf-v1.2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4744747 (4.5M) [text/plain]
Saving to: ‘data/piaf-v1.2.json.1’

piaf-v1.2.json.1    100%[===================>]   4.52M  --.-KB/s    in 0.02s   

2022-06-04 19:53:46 (228 MB/s) - ‘data/piaf-v1.2.json.1’ saved [4744747/4744747]



In [8]:
# #### 1.1节： 文本的读取
# 把给定的文本数据读入到```qlist```和```alist```当中，这两个分别是列表，其中```qlist```是问题的列表，```alist```是对应的答案列表
def read_corpus():
    """
    读取给定的语料库，并把问题列表和答案列表分别写入到 qlist, alist 里面。 在此过程中，不用对字符换做任何的处理（这部分需要在 Part 2.3里处理）
    qlist = ["问题1"， “问题2”， “问题3” ....]
    alist = ["答案1", "答案2", "答案3" ....]
    务必要让每一个问题和答案对应起来（下标位置一致）
    """
    # 问题列表
    qlist = []
    # 答案列表
    alist = []
    # 加载json文件
    # 加载json文件
    filename = 'data/piaf-v1.2.json'
    datas = json.load(open(filename, 'r'))
    data = datas['data']
    print(datas)
    for d in data:
        paragraph = d['paragraphs']
        for p in paragraph:
            qas = p['qas']
            for qa in qas:
                # 处理is_impossible为True时answers空
                # if (not qa['is_impossible']):
                qlist.append(qa['question'])
                alist.append(qa['answers'][0]['text'])
                  
    # 如果它的条件返回错误，则终止程序执行
    # 这行代码是确保长度一样
    assert len(qlist) == len(alist)
    return qlist, alist

In [9]:
# 读取给定的语料库，并把问题列表和答案列表分别写入到 qlist, alist
qlist, alist = read_corpus()

# ### 1.2 理解数据（可视化分析/统计信息）
# 统计一下在qlist中总共出现了多少个单词？ 总共出现了多少个不同的单词(unique word)？
# 这里需要做简单的分词，对于英文我们根据空格来分词即可，其他过滤暂不考虑（只需分词）
words_qlist = dict()
for q in qlist:
    # 以空格为分词，都转为小写
    words = q.strip().split(' ')
    for w in words:
        if w.lower() in words_qlist:
            words_qlist[w.lower()] += 1
        else:
            words_qlist[w.lower()] = 1
word_total = len(words_qlist)
print("qlist的单词统计：",word_total)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



qlist的单词统计： 13783


In [10]:
# #### 1.3 文本预处理
# 此部分需要做文本方面的处理。 以下是可以用到的一些方法：
#
# - 1. 停用词过滤 （去网上搜一下 "english stop words list"，会出现很多包含停用词库的网页，或者直接使用NLTK自带的）
# - 2. 转换成lower_case： 这是一个基本的操作
# - 3. 去掉一些无用的符号： 比如连续的感叹号！！！， 或者一些奇怪的单词。
# - 4. 去掉出现频率很低的词：比如出现次数少于10,20.... （想一下如何选择阈值）
# - 5. 对于数字的处理： 分词完只有有些单词可能就是数字比如44，415，把所有这些数字都看成是一个单词，这个新的单词我们可以定义为 "#number"
# - 6. lemmazation： 在这里不要使用stemming， 因为stemming的结果有可能不是valid word。

In [11]:
import nltk
from nltk.corpus import stopwords
import codecs
import re


# 去掉一些无用的符号
def tokenizer(ori_list):
    # 利用正则表达式去掉无用的符号
    # compile 函数用于编译正则表达式，[]用来表示一组字符
    # \s匹配任意空白字符，等价于 [\t\n\r\f]。
    SYMBOLS = re.compile('[\s;\"\",.!?\\/\[\]\{\}\(\)-]+')
    new_list = []
    for q in ori_list:
        # split 方法按照能够匹配的子串将字符串分割后返回列表
        words = SYMBOLS.split(q.lower().strip())
        new_list.append(' '.join(words))
    return new_list

In [12]:
# 去掉question的停用词
def removeStopWord(ori_list):
    new_list = []
    # nltk中stopwords包含what等，但是在QA问题中，这算关键词，所以不看作关键词
    restored = ['que', 'qui', 'quoi', 'où', 'comment', 'pourquoi','combien','quel','quels','quelle','quelles']
    # nltk中自带的停用词库
    french_stop_words = list(
        set(stopwords.words('french')))  
    # 将在QA问答系统中不算停用词的词去掉
    for w in restored:
      if w in french_stop_words:
          french_stop_words.remove(w)
      else:
          pass
    for q in ori_list:
        # 将每个问句的停用词去掉
        sentence = ' '.join([w for w in q.strip().split(' ') if w not in french_stop_words])
        # 将去掉停用词的问句添加至列表中
        new_list.append(sentence)
    return new_list

In [13]:
def removeLowFrequence(ori_list, vocabulary, thres=10):
    """
    去掉低频率的词
    :param ori_list: 预处理后的问题列表
    :param vocabulary: 词频率字典
    :param thres: 频率阈值，可以基于数据实际情况进行调整
    :return: 新的问题列表
    """
    # 根据thres筛选词表，小于thres的词去掉
    new_list = []
    for q in ori_list:
        sentence = ' '.join([w for w in q.strip().split(' ') if vocabulary[w] >= thres])
        new_list.append(sentence)
    return new_list

In [14]:
def replaceDigits(ori_list, replace='#number'):
    """
    将数字统一替换为replace,默认#number
    :param ori_list: 预处理后的问题列表
    :param replace:
    :return:
    """
    # 编译正则表达式：匹配1个或多个数字
    DIGITS = re.compile('\d+')
    new_list = []
    for q in ori_list:
        # re.sub用于替换字符串中的匹配项，相当于在q中查找连续的数字替换为#number
        q = DIGITS.sub(replace, q)
        # 将处理后的问题字符串添加到新列表中
        new_list.append(q)
    return new_list

In [15]:
def createVocab(ori_list):
    """
    创建词表，统计所有单词总数与每个单词总数
    :param ori_list:预处理后的列表
    :return:所有单词总数与每个单词总数
    """
    count = 0
    vocab_count = dict()
    for q in ori_list:
        words = q.strip().split(' ')
        count += len(words)
        for w in words:
            if w in vocab_count:
                vocab_count[w] += 1
            else:
                vocab_count[w] = 1
    return vocab_count, count

In [16]:
def writeFile(oriList, filename):
    """
    将处理后的问题列表写入到文件中
    :param oriList: 预处理后的问题列表
    :param filename: 文件名
    """
    with codecs.open(filename, 'w', 'utf8') as Fout:
        for q in oriList:
            Fout.write(q + u'\n')

In [17]:
def writeVocab(vocabulary, filename):
    """
    将词表写入到文件中
    :param vocabulary: 词表
    :param filename: 文件名
    """
    sortedList = sorted(vocabulary.items(), key=lambda d: d[1])
    with codecs.open(filename, 'w', 'utf8') as Fout:
        for (w, c) in sortedList:
            Fout.write(w + u':' + str(c) + u'\n')

In [20]:
import nltk
nltk.download("stopwords")
# 去掉一些无用的符号
new_list = tokenizer(qlist)
# 停用词过滤
new_list = removeStopWord(new_list)
# 数字处理-将数字替换为#number
new_list = replaceDigits(new_list)
# 创建词表并统计所有单词数目
vocabulary, count = createVocab(new_list)
# 去掉低频率的词
new_list = removeLowFrequence(new_list, vocabulary, 5)
# 重新统计词频
vocab_count, count = createVocab(new_list)
# 将词表写入到文件“train.vocab”中
writeVocab(vocab_count, "train.vocab")
qlist = new_list
print(qlist[:100])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['combien personnes ministère', 'combien', 'quel part budget', 'quel montant #number', 'quel montant #number', 'qui a', 'que trouve sous mont', 'qui hors commun', 'a lieu quel moment', 'qui a fait', 'qui constitution', 'qui grand père', 'parents personnes', 'qui apprendre', "comment s'appelait a #number ans", 'quelle année tombe', 'qui mort', 'quel métier nicolas', 'que doit avant derniers', 'pourquoi', 'quand a thèse', 'a quel âge obtient', 'quelles études suit guillaume', 'a quel âge guillaume marie andré part paris', 'combien temps a duré', 'comment nil nuit', 'comment nil', 'quel mois froid nil', 'comment nil', "comment l'air nil", "quel pays surnommé l'afrique", 'quelle croissance moyenne années #number', 'qui vice parlement #number #number', 'comment', 'quelle proportion femmes parlement #number', 'quand guerre', 'quel pays passe accord #number', 'quelle institution fait

In [ ]:
# ### 第二部分： 文本的表示
# 文本处理完之后，我们需要做文本表示，这里有3种方式：
#
# - 1. 使用```tf-idf vector```
# - 2. 使用embedding技术如```word2vec```, ```bert embedding```等

# #### 2.1 使用tf-idf表示向量
# 把```qlist```中的每一个问题的字符串转换成```tf-idf```向量, 转换之后的结果存储在```X```矩阵里。
# # ``X``的大小是： ``N* D``的矩阵。 这里``N``是问题的个数（样本个数），``D``是词典库的大小


In [21]:
import numpy as np


def computeTF(vocab, c):
    """
    计算每个词的词频
    :param vocab: 词频字典:键是单词，值是所有问句中单词出现的次数
    :param c: 单词总数
    :return: TF
    """
    # 初始化TF
    TF = np.ones(len(vocab))
    # 词频字典
    word2id = dict()
    # 单词字典
    id2word = dict()
    for word, fre in vocab.items():
        # 计算TF值：每个单词出现的个数/总的单词个数
        TF[len(word2id)] = 1.0 * fre / c
        id2word[len(word2id)] = word
        word2id[word] = len(word2id)
    return TF, word2id, id2word

In [22]:
def computeIDF(word2id, qlist):
    """
    计算IDF：log[问句总数/(包含单词t的问句总数+1)]
    :param word2id:单词字典
    :param qlist:问句列表
    :return:
    """
    IDF = np.ones(len(word2id))
    for q in qlist:
        # 去重
        words = set(q.strip().split())
        for w in words:
            # 统计单词出现在问句中的总数
            IDF[word2id[w]] += 1
    # 计算IDF
    IDF /= len(qlist)
    IDF = -1.0 * np.log2(IDF)
    return IDF

In [23]:
def computeSentenceEach(sentence, tfidf, word2id):
    """
    给定句子，计算句子TF-IDF,tfidf是一个1*M的矩阵,M为词表大小
    :param sentence:句子
    :param tfidf:TF-IDF向量
    :param word2id:词表
    :return:
    """
    sentence_tfidf = np.zeros(len(word2id))
    # 将问句以空格进行分割
    for w in sentence.strip().split(' '):
        if w not in word2id:
            continue
        # 碰到在词表word2id中的单词
        sentence_tfidf[word2id[w]] = tfidf[word2id[w]]
    return sentence_tfidf

In [24]:
def computeSentence(qlist, word2id, tfidf):
    """
    把```qlist```中的每一个问题的字符串转换成```tf-idf```向量, 转换之后的结果存储在```X```矩阵里
    :param qlist: 问题列表
    :param word2id: 词表(字典形式)
    :param tfidf: TF-IDF(与词表中的键一一对应)
    :return: X矩阵
    """
    # 对所有句子分别求tfidf
    X_tfidf = np.zeros((len(qlist), len(word2id)))
    for i, q in enumerate(qlist):
        X_tfidf[i] = computeSentenceEach(q, tfidf, word2id)
        # print(X_tfidf[i])
    return X_tfidf

In [25]:
# 计算每个词的TF,词表字典
TF, word2id, id2word = computeTF(vocab_count, count)
# 计算IDF：log[问句总数/(包含单词t的问句总数+1)]
IDF = computeIDF(word2id, qlist)
# 用TF，IDF计算最终的tf-idf,定义一个tf-idf的vectorizer
vectorizer = np.multiply(TF, IDF)
# 把```qlist```中的每一个问题的字符串转换成```tf-idf```向量, 转换之后的结果存储在```X```矩阵里
X_tfidf = computeSentence(qlist, word2id, vectorizer)

In [26]:
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 6.7 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149858 sha256=4522a82c1dfc5677ceeef21ccd7c076687ef8bcbd71fb9ee27a89aadf4efab0f
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
# #### 2.2 使用wordvec + average pooling
# 词向量方面需要下载： https://nlp.stanford.edu/projects/glove/ （请下载``glove.6B.zip``），并使用``d=200``的词向量（200维）。
# 国外网址如果很慢，可以在百度上搜索国内服务器上的。 每个词向量获取完之后，即可以得到一个句子的向量。 我们通过``average pooling``来实现句子的向量。
# 基于Glove向量获取句子向量
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip
model_path = '/content/data/glove.6B.200d.txt'

In [ ]:
def loadEmbedding(filename):
    """
    加载glove模型，转化为word2vec，再加载到word2vec模型
    这两种模型形式上是一样的，在数据的保存形式上有略微的差异
    :param filename: glove文件
    :return: word2vec模型
    """
    word2vec_temp_file = get_tmpfile("test_word2vec.txt")
    # 加载glove模型，转化为word2vec
    glove2word2vec(filename, word2vec_temp_file)
    # 再加载到word2vec模型
    model = KeyedVectors.load_word2vec_format(word2vec_temp_file)
    return model

In [27]:
!wget -P data https://git.unistra.fr/dbernhard/aisd_d3/-/raw/master/data/fra_newscrawl-public_2019_300K-tokenised-sentences.txt

--2022-06-04 19:56:07--  https://git.unistra.fr/dbernhard/aisd_d3/-/raw/master/data/fra_newscrawl-public_2019_300K-tokenised-sentences.txt
Resolving git.unistra.fr (git.unistra.fr)... 130.79.254.48
Connecting to git.unistra.fr (git.unistra.fr)|130.79.254.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38143498 (36M) [text/plain]
Saving to: ‘data/fra_newscrawl-public_2019_300K-tokenised-sentences.txt’

fra_newscrawl-publi 100%[===================>]  36.38M  19.1MB/s    in 1.9s    

2022-06-04 19:56:10 (19.1 MB/s) - ‘data/fra_newscrawl-public_2019_300K-tokenised-sentences.txt’ saved [38143498/38143498]



In [28]:
import logging

from gensim.models.word2vec import Word2Vec
from gensim.models.word2vec import LineSentence
import pandas as pd

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)
# Pour mieux afficher les textes (colonnes plus larges)
pd.set_option('display.max_colwidth', 600) 

In [57]:
input_corpus = LineSentence('data/fra_newscrawl-public_2019_300K-tokenised-sentences.txt')
# Dimension des vecteurs
VEC_SIZE = 300
# Taille de la fenêtre de contexte
WINDOW_SIZE = 5
# Nombre minimal d'occurrences pour les mots
MIN_FREQ = 1
# Nombre d'itérations sur le corpus complet
NB_EPOCHS = 10
# Nombre de threads (fils d'exécution) pour accélérer l'apprentissage
NB_WORKERS = 4
# Nombre de mots « négatifs » à sélectionner pour l'échantillonnage négatif
NEGATIVE_SAMPLE = 5
# Seuil pour sélectionner les mots fréquents qui sont sous-échantillonnés
SUB_SAMPLING = 0.00001
# Taux d'apprentissage (learning rate) initial : contrôle l'amplitude des ajustements effectués sur les poids du réseau
LR = 0.025
# Valeur minimale pour le taux d'apprentissage
MIN_LR = 0.0001
# Utilisation de Skip-gram
SG = 1

In [58]:
model = Word2Vec(sentences=input_corpus, 
                 size=VEC_SIZE, window=WINDOW_SIZE,
                 min_count=MIN_FREQ, iter=NB_EPOCHS,
                 sg=SG, workers=NB_WORKERS,
                 negative=NEGATIVE_SAMPLE,
                 sample=SUB_SAMPLING, alpha=LR,
                 min_alpha=MIN_LR, compute_loss=True)

2022-06-04 20:24:03,005 : INFO : collecting all words and their counts
2022-06-04 20:24:03,008 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-06-04 20:24:03,145 : INFO : PROGRESS: at sentence #10000, processed 252527 words, keeping 30145 word types
2022-06-04 20:24:03,260 : INFO : PROGRESS: at sentence #20000, processed 503173 words, keeping 44394 word types
2022-06-04 20:24:03,374 : INFO : PROGRESS: at sentence #30000, processed 728801 words, keeping 55092 word types
2022-06-04 20:24:03,474 : INFO : PROGRESS: at sentence #40000, processed 956669 words, keeping 62463 word types
2022-06-04 20:24:03,579 : INFO : PROGRESS: at sentence #50000, processed 1172099 words, keeping 68513 word types
2022-06-04 20:24:03,690 : INFO : PROGRESS: at sentence #60000, processed 1406090 words, keeping 76743 word types
2022-06-04 20:24:03,803 : INFO : PROGRESS: at sentence #70000, processed 1662992 words, keeping 84206 word types
2022-06-04 20:24:03,922 : INFO : PROGRESS: 

In [76]:
def computeGloveSentenceEach(sentence, embedding):
    """
    :param sentence:问题
    :param embedding:wordvec模型
    :return:
    """
    # 查找句子中每个词的embedding,将所有embedding进行加和求均值
    emb = np.zeros(300)
    words = sentence.strip().split(' ')
    for w in words:
      # 如果单词是新词，则重命名为'unknown'
      if w not in embedding:
            # 没有lookup的即为unknown
          w = 'unknown'
      else:
        pass
        # 将所有embedding进行加和求均值
        emb += embedding.wv.get_vector(w)
        # emb += embedding[w]
    return emb / len(words)

In [77]:
def computeGloveSentence(qlist, embedding):
    """
    对每一个句子来构建句子向量
    :param qlist:问题列表
    :param embedding:word2vec模型
    :return:
    """
    # 对每一个句子进行求均值的embedding
    X_w2v = np.zeros((len(qlist), 300))
    for i, q in enumerate(qlist):
        # 编码每一个问题
        X_w2v[i] = computeGloveSentenceEach(q, embedding)
        # print(X_w2v)
    return X_w2v

In [78]:
# 加载到word2vec模型
# 这是 D*H的矩阵，这里的D是词典库的大小， H是词向量的大小。 这里面我们给定的每个单词的词向量，
emb = model.wv
# 初始化完emb之后就可以对每一个句子来构建句子向量了，这个过程使用average pooling来实现
X_w2v = computeGloveSentence(qlist, emb)
print(X_w2v)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


[[-0.03965362  0.24082079 -0.16425829 ... -0.00299822 -0.1400892
  -0.22496009]
 [-0.04005026 -0.0109747  -0.12546748 ...  0.07687889 -0.17839265
  -0.09179237]
 [-0.06666278  0.10480606 -0.07851283 ... -0.0126547  -0.09636211
  -0.07799855]
 ...
 [-0.03554737  0.07470422  0.00846837 ...  0.02636478 -0.04205069
  -0.10601449]
 [-0.07434175  0.1010364   0.00940921 ...  0.01630229 -0.03028429
  -0.20319642]
 [-0.0262852   0.09887614 -0.03757961 ...  0.05532637 -0.08545647
  -0.13127398]]


In [79]:
!pip install bert_embedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.8 MB 25.1 MB/s 
     |████████████████████████████████| 209 kB 65.8 MB/s 
     |████████████████████████████████| 29.6 MB 1.5 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.6.0-py3-none-any.whl size=259930 sha256=3ca603b3ade505b40b49441739471cccee3d89b0be0b07381593183885bcbb45
  Stored in directory: /root/.cache/pip/wheels/a6/41/8f/45bd1c58055d87aee5a71b6756a427ea8d92e506b3a9d17370
Successfully built gluonnlp
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

In [ ]:
'''
# TODO 基于BERT的句子向量计算
from bert_embedding import BertEmbedding

sentence_embedding = np.ones((len(qlist), 768))
# 加载Bert模型，model，dataset_name,须指定
bert_embedding = BertEmbedding(model='bert_12_768_12', dataset_name='wiki_multilingual_cased')
# 查询所有句子的Bert  embedding
all_embedding = bert_embedding(qlist, 'sum')
for q in qlist:
    for i in range(len(all_embedding)):
        print(all_embedding[i][1])
        # 将每一个句子中每一个词的向量进行拼接求平均得到句子向量
        sentence_embedding[i] = np.sum(all_embedding[i][1], axis=0) / len(q.strip().split(' '))
        if i == 0:
            print(sentence_embedding[i])
'''

In [ ]:
# 每一个句子的向量结果存放在X_bert矩阵里。行数为句子的总个数，列数为一个句子embedding大小。
# X_bert = sentence_embedding

In [80]:
# ### 第三部分： 相似度匹配以及搜索
# 在这部分里，我们需要把用户每一个输入跟知识库里的每一个问题做一个相似度计算，从而得出最相似的问题。但对于这个问题，时间复杂度其实很高，所以我们需要结合倒排表来获取相似度最高的问题，从而获得答案。

# In[ ]:


# #### 3.1 tf-idf + 余弦相似度
# 我们可以直接基于计算出来的``tf-idf``向量，计算用户最新问题与库中存储的问题之间的相似度，从而选择相似度最高的问题的答案。
# 这个方法的复杂度为``O(N)``， ``N``是库中问题的个数。

import queue as Q

# 优先级队列实现大顶堆Heap,每次输出都是相似度最大值
que = Q.PriorityQueue()

In [81]:
def cosineSimilarity(vec1, vec2):
    # 定义余弦相似度,余弦相似度越大，两个向量越相似
    return np.dot(vec1, vec2.T) / (np.sqrt(np.sum(vec1 ** 2)) * np.sqrt(np.sum(vec2 ** 2)))


def get_top_results_tfidf_noindex(query):
    """
    给定用户输入的问题 query, 返回最有可能的TOP 5问题。
    :param query:用户新问题
    :return:
    """
    """
    给定用户输入的问题 query, 返回最有可能的TOP 5问题。这里面需要做到以下几点：
    1. 对于用户的输入 query 首先做一系列的预处理(上面提到的方法)，然后再转换成tf-idf向量（利用上面的vectorizer)
    2. 计算跟每个库里的问题之间的相似度
    3. 找出相似度最高的top5问题的答案
    """
    top = 5
    # 将用户输入的新问题用tf-idf来表示
    query_tfidf = computeSentenceEach(query.lower(), vectorizer, word2id)
    for i, vec in enumerate(X_tfidf):
        # 计算原问题与用户输入的新问题的相似度
        result = cosineSimilarity(vec, query_tfidf)
        # print(result)
        # 存放到大顶堆里面
        que.put((-1 * result, i))
    i = 0
    top_idxs = []
    while (i < top and not que.empty()):
        # top_idxs存放相似度最高的（存在qlist里的）问题的下标
        top_idxs.append(que.get()[1])
        i += 1
    print(top_idxs)
    # 返回相似度最高的问题对应的答案，作为TOP5答案
    return np.array(alist)[top_idxs]

In [88]:
# 给定用户输入的问题 query, 返回最有可能的TOP 5问题的答案。
results = get_top_results_tfidf_noindex("comment les africains répandent la religion")
print(" ; ".join(results))

[2275, 1537, 658, 739, 843]
en commerçant ; empruntent des mots français ; Paris ; la prochaine suppression des Chambres de l’édit de Paris et de Rouen ; protestations d'attachement éternel, serments non tenus, engagements réciproques, alliances familiales de même rang, ventes et achats de terrains, marchandages continuels et contrats 


In [89]:
word_doc = dict()
# key:word,value:包含该词的句子序号的列表
for i, q in enumerate(qlist):
    words = q.strip().split(' ')
    for w in set(words):
        if w not in word_doc:
            # 没在word_doc中的，建立一个空listi
            word_doc[w] = set([])
        word_doc[w] = word_doc[w] | set([i])

# 定一个一个简单的倒排表，是一个map结构。 循环所有qlist一遍就可以
inverted_idx = word_doc

In [90]:
# #### 3.3 语义相似度
# 读取语义相关的单词
import codecs


def get_related_words(filename):
    """
    从预处理的相似词的文件加载相似词信息
    文件格式w1 w2 w3..w11,其中w1为原词，w2-w11为w1的相似词
    :param filename: 文件名
    :return:
    """
    related_words = {}
    with codecs.open(filename, 'r', 'utf8') as Fin:
        lines = Fin.readlines()
    for line in lines:
        words = line.strip().split(' ')
        # 键为原词，值为相似词
        related_words[words[0]] = words[1:]
    return related_words

In [ ]:
# 从预处理的相似词的文件加载相似词信息
related_words = get_related_words('data/related_words.txt')

In [ ]:
# #### 3.4 利用倒排表搜索
# 在这里，我们使用倒排表先获得一批候选问题，然后再通过余弦相似度做精准匹配，这样一来可以节省大量的时间。搜索过程分成两步：
# - 使用倒排表把候选问题全部提取出来。首先，对输入的新问题做分词等必要的预处理工作，然后对于句子里的每一个单词，从``related_words``里提取出跟它意思相近的top 10单词， 然后根据这些top词从倒排表里提取相关的文档，把所有的文档返回。 这部分可以放在下面的函数当中，也可以放在外部。
# - 然后针对于这些文档做余弦相似度的计算，最后排序并选出最好的答案。

In [95]:
import queue as Q


def cosineSimilarity(vec1, vec2):
    # 定义余弦相似度
    return np.dot(vec1, vec2.T) / (np.sqrt(np.sum(vec1 ** 2)) * np.sqrt(np.sum(vec2 ** 2)))


def getCandidate(query):
    """
    根据查询句子中每个词及其10个相似词所在的序号列表，求交集
    :param query: 问题
    :return:
    """
    searched = set()
    for w in query.strip().split(' '):
        # 如果单词不在word2id中或者不在倒排表中
        if w not in word2id or w not in inverted_idx:
            continue
        # 搜索原词所在的序号列表
        if len(searched) == 0:
            searched = set(inverted_idx[w])
        else:
            searched = searched & set(inverted_idx[w])
            '''
        # 搜索相似词所在的列表
        if w in related_words:
            for similar in related_words[w]:
                searched = searched & set(inverted_idx[similar])
                '''
    return searched

In [96]:
def get_top_results_tfidf(query):
    """
    基于TF-IDF,给定用户输入的问题 query, 返回最有可能的TOP 5问题。
    这里面需要做到以下几点：
    1. 利用倒排表来筛选 candidate （需要使用related_words).
    2. 对于候选文档，计算跟输入问题之间的相似度
    3. 找出相似度最高的top5问题的答案
    """
    top = 5
    # 计算给定句子的TF-IDF
    query_tfidf = computeSentenceEach(query, vectorizer, word2id)
    # 优先级队列实现大顶堆Heap,每次输出都是相似度最大值
    results = Q.PriorityQueue()
    # 利用倒排索引表获取相似问题
    searched = getCandidate(query)
    # print(len(searched))
    for candidate in searched:
        # 计算candidate与query的余弦相似度
        result = cosineSimilarity(query_tfidf, X_tfidf[candidate])
        # 优先级队列中保存相似度和对应的candidate序号
        # -1保证降序
        results.put((-1 * result, candidate))
    i = 0
    # top_idxs存放相似度最高的（存在qlist里的）问题的索引
    top_idxs = []
    # hint: 利用priority queue来找出top results.
    while i < top and not results.empty():
        top_idxs.append(results.get()[1])
        i += 1
    # 返回相似度最高的问题对应的答案，作为TOP5答案
    return np.array(alist)[top_idxs]

In [97]:
def get_top_results_w2v(query):
    """
    基于word2vec，给定用户输入的问题 query, 返回最有可能的TOP 5问题。
    这里面需要做到以下几点：
    1. 利用倒排表来筛选 candidate （需要使用related_words).
    2. 对于候选文档，计算跟输入问题之间的相似度
    3. 找出相似度最高的top5问题的答案
    """
    # embedding用glove
    top = 5
    # 利用glove将问题进行编码
    query_emb = computeGloveSentenceEach(query, emb)
    # 优先级队列实现大顶堆Heap,每次输出都是相似度最大值
    results = Q.PriorityQueue()
    # 利用倒排索引表获取相似问题
    searched = getCandidate(query)
    for candidate in searched:
        # 计算candidate与query的余弦相似度
        result = cosineSimilarity(query_emb, X_w2v[candidate])
        # 优先级队列中保存相似度和对应的candidate序号
        # -1保证降序
        results.put((-1 * result, candidate))
    # top_idxs存放相似度最高的（存在qlist里的）问题的索引
    top_idxs = []
    i = 0
    # hint: 利用priority queue来找出top results.
    while i < top and not results.empty():
        top_idxs.append(results.get()[1])
        i += 1
    # 返回相似度最高的问题对应的答案，作为TOP5答案
    return np.array(alist)[top_idxs]

In [98]:
def get_top_results_bert(query):
    """
    给定用户输入的问题 query, 返回最有可能的TOP 5问题。
    这里面需要做到以下几点：
    1. 利用倒排表来筛选 candidate （需要使用related_words).
    2. 对于候选文档，计算跟输入问题之间的相似度
    3. 找出相似度最高的top5问题的答案
    """
    top = 5
    # embedding用Bert embedding
    query_emb = np.sum(bert_embedding([query], 'sum')[0][1], axis=0) / len(query.strip().split())
    # 优先级队列实现大顶堆Heap,每次输出都是相似度最大值
    results = Q.PriorityQueue()
    # 利用倒排索引表获取相似问题
    searched = getCandidate(query)
    for candidate in searched:
        # 计算candidate与query的余弦相似度
        result = cosineSimilarity(query_emb, X_bert[candidate])
        # 优先级队列中保存相似度和对应的candidate序号
        # -1保证降序
        results.put((-1 * result, candidate))
    # top_idxs存放相似度最高的（存在qlist里的）问题的索引
    top_idxs = []
    i = 0
    # hint: 利用priority queue来找出top results.
    while i < top and not results.empty():
        top_idxs.append(results.get()[1])
        i += 1
    # 返回相似度最高的问题对应的答案，作为TOP5答案
    return np.array(alist)[top_idxs]


In [105]:
!pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.7 MB 34.0 MB/s 


In [106]:
# correcteur en français avec spellchecker
from spellchecker import SpellChecker

spell = SpellChecker(language='fr')

In [108]:
# la quesiton mal orthographié
query = 'commnet les afrcains répandent la religoin'
# tokens
misspelled = query.split(' ')
# correcter mot par mot
correcte = [str(spell.correction(word)) for word in misspelled]
# retour la question à l'orthographe correcte
query = " ".join(correcte)
print("la question correcte : ",query)

results_tfidf_noindex = get_top_results_tfidf_noindex(query)
results_tfidf = get_top_results_tfidf(query)
results_w2v = get_top_results_w2v(query)
# results_bert = get_top_results_bert(query)

print("tfidf_noindex : "," ; ".join(results_tfidf_noindex))
print("tf-idf : "," ; ".join(results_tfidf))
print("word2vec : "," ; ".join(results_w2v))
# print("bert : "," ; ".join(results_bert))

la question correcte :  comment les africains répandent la religion
[2275, 1537, 8337, 8552, 8559]
tfidf_noindex :  en commerçant ; empruntent des mots français ; historienne des mathématiques ; vol ; ingénierie de fiabilité
tf-idf :  en commerçant
word2vec :  en commerçant


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
